# Biostat M280 Homework 4
Sarah Ji

**Due June 12 @ 11:59PM**

In this homework, we build a classifer for handwritten digit recognition. Following figure shows example bitmaps of handwritten digits from U.S. postal envelopes. 

Each digit is represented by a $32 \times 32$ bitmap in which each element indicates one pixel with a value of white or black. Each $32 \times 32$ bitmap is divided into blocks of $4 \times 4$, and the number of white pixels are counted in each block. Therefore each handwritten digit is summarized by a vector $\mathbf{x} = (x_1, \ldots, x_{64})$ of length 64 where each element is a count between 0 and 16. 

We will use a model-based method by assuming a distribution on the count vector and carry out classification using probabilities. A common distribution for count vectors is the multinomial distribution. However as you will see in Q10, it is not a good model for handwritten digits. Let's work on a more flexible model for count vectors. In the Dirichlet-multinomial model, we assume the multinomial probabilities $\mathbf{p} = (p_1,\ldots, p_d)$ follow a Dirichlet distribution with parameter vector $\alpha = (\alpha_1,\ldots, \alpha_d)$, $\alpha_j>0$, and density
$$
\begin{eqnarray*}
	\pi(\mathbf{p}) =  \frac{\Gamma(|\alpha|)}{\prod_{j=1}^d \Gamma(\alpha_j)} \prod_{j=1}^d p_j^{\alpha_j-1},
\end{eqnarray*} 
$$
where $|\alpha|=\sum_{j=1}^d \alpha_j$.

## Q1

For a multivariate count vector $\mathbf{x}=(x_1,\ldots,x_d)$ with batch size $|\mathbf{x}|=\sum_{j=1}^d x_j$, show that the probability mass function for Dirichlet-multinomial distribution is
$$
    f(\mathbf{x} \mid \alpha) 
	= \int_{\Delta_d} \binom{|\mathbf{x}|}{\mathbf{x}} \prod_{j=1}^d p_j^{x_j} \pi(\mathbf{p}) \, d \mathbf{p}  
    = \binom{|\mathbf{x}|}{\mathbf{x}} \frac{\prod_{j=1}^d \Gamma(\alpha_j+x_j)}{\prod_{j=1}^d \Gamma(\alpha_j)} \frac{\Gamma(|\alpha|)}{\Gamma(|\alpha|+|\mathbf{x}|)}
$$
where $\Delta_d$ is the unit simplex in $d$ dimensions and $|\alpha| = \sum_{j=1}^d \alpha_j$.


## Q1 Solution: 
 
>First we should plug in the pi(p) formula in the f(x) formula and then combine like terms.

 $$
\int_{\Delta_d} \binom{|\mathbf{x}|}{\mathbf{x}} \prod_{j=1}^d p_j^{x_j} \pi(\mathbf{p}) \, d \mathbf{p} 
= \int_{\Delta_d}\binom{|\mathbf{x}|}{\mathbf{x}} \prod_{j=1}^d p_j^{x_j}\frac{\Gamma(|\alpha|)}{\prod_{j=1}^d \Gamma(\alpha_j)} \prod_{j=1}^d p_j^{\alpha_j-1}d \mathbf{p}
$$
 
>Then we can use the kernel trick to integrate the first pdf to 1 and change the coefficients accordingly.
 

$$
=\binom{|\mathbf{x}|}{\mathbf{x}}\frac{\Gamma(|\alpha|)}{\prod_{j=1}^d \Gamma(\alpha_j)} \int_{\Delta_d} \prod_{j=1}^d p_j^{x_j + {\alpha_j-1}} d\mathbf{p}
$$
$$
=\binom{|\mathbf{x}|}{\mathbf{x}}\frac{\Gamma(|\alpha|)}{\prod_{j=1}^d \Gamma(\alpha_j)} \int_{\Delta_d} \prod_{j=1}^d p_j^{x_j + {\alpha_j-1}} \frac{\prod_{j=1}^d \Gamma(\alpha_j+x_j)}{\prod_{j=1}^d \Gamma(\alpha_j+x_j)}\frac{\Gamma(|\alpha|+|\mathbf{x}|)}{\Gamma(|\alpha|+|\mathbf{x}|)}d\mathbf{p}
$$

$$
=\binom{|\mathbf{x}|}{\mathbf{x}}\frac{\Gamma(|\alpha|)}{\prod_{j=1}^d \Gamma(\alpha_j)} \frac{\prod_{j=1}^d \Gamma(\alpha_j+x_j)}{\Gamma(|\alpha|+|\mathbf{x}|)}\int_{\Delta_d} \prod_{j=1}^d p_j^{x_j + {\alpha_j-1}} \frac{\prod_{j=1}^d \Gamma(\alpha_j+x_j)}{\Gamma(|\alpha|+|\mathbf{x}|)}d\mathbf{p}
$$

> Note this is a density of a Dirchlet pdf and so the integral evalutes to 1, giving us: 

$$
=\binom{|\mathbf{x}|}{\mathbf{x}}\frac{\Gamma(|\alpha|)}{\prod_{j=1}^d \Gamma(\alpha_j)} \frac{\prod_{j=1}^d \Gamma(\alpha_j+x_j)}{\Gamma(|\alpha|+|\mathbf{x}|)} 1
$$
$$
=\binom{|\mathbf{x}|}{\mathbf{x}} \frac{\prod_{j=1}^d \Gamma(\alpha_j+x_j)}{\prod_{j=1}^d \Gamma(\alpha_j)} \frac{\Gamma(|\alpha|)}{\Gamma(|\alpha|+|\mathbf{x}|)}
$$

$$
= f(\mathbf{x} \mid \alpha)
$$

## Q2

Given independent data points $\mathbf{x}_1, \ldots, \mathbf{x}_n$, show that the log-likelihood is
$$
L(\alpha) = \sum_{i=1}^n \ln \binom{|\mathbf{x}_i|}{\mathbf{x}_i} + \sum_{i=1}^n \sum_{j=1}^d [\ln \Gamma(\alpha_j + x_{ij}) - \ln \Gamma(\alpha_j)] - \sum_{i=1}^n [\ln \Gamma(|\alpha|+|\mathbf{x}_i|) - \ln \Gamma(|\alpha|)].
$$
Is the log-likelihood a concave function?

## Q2 Solution:

The log likelihood is neither a concave function nor a convex function. Tricky, tricky!

## Q3

Write Julia function to compute the log-density of the Dirichlet-multinomial distribution.

## Q3 Solution:

Below, I have written a Julia function to compute the log probability density function of the Dirichlet-Multinomial Distribution.

In [1]:
using SpecialFunctions
"""
    dirmult_logpdf(x::Vector, α::Vector)
    
Compute the log-pdf of Dirichlet-multinomial distribution with parameter `α` 
at data point `x`.
"""
function dirmult_logpdf(x::Vector, α::Vector)
    term1 = 0.0
    Logpdf = 0.0
    sumalpha = sum(α)
    sumx = sum(x)
    for i in 1:length(x)
        if α[i] == 0 && x[i] == 0 # we have to hard code these values
            Logpdf += 0
        elseif α[i] == 0 && x[i] > 0
            Logpdf += - Inf # we have to hard code these values
        else
        term1 += lgamma(α[i] + x[i]) - lgamma(α[i]) - lfact(x[i])
        Logpdf = term1 + lfact(sumx) + lgamma(sumalpha) - lgamma(sumx + sumalpha)
    end
    end
    return Logpdf
end

function dirmult_logpdf!(r::Vector, X::Matrix, α::Vector)
    for j in 1:size(X, 2)
        r[j] = dirmult_logpdf(X[:, j], α)
    end
    return r
end

"""
    dirmult_logpdf(X, α)
    
Compute the log-pdf of Dirichlet-multinomial distribution with parameter `α` 
at each data point in `X`. Each column of `X` is one data point.
"""
function dirmult_logpdf(X::Matrix, α::Vector)
    r = zeros(size(X, 2))
    dirmult_logpdf!(r, X, α)
end

dirmult_logpdf

## Q4

Read in `optdigits.tra`, the training set of 3823 handwritten digits. Each row contains the 64 counts of a digit and the last element (65th element) indicates what digit it is. For grading purpose, evaluate the total log-likelihood of this data at parameter values $\alpha=(1,\ldots,1)$ using your function in Q3.

## Q4 Solution:

In [66]:
traindata = readcsv("optdigits.tra", Int)
X = copy(traindata[:, 1:64])

3823×64 Array{Int64,2}:
 0  1   6  15  12   1   0  0  0  7  16  …   0  0  0  0   6  14   7   1   0  0
 0  0  10  16   6   0   0  0  0  7  16      3  0  0  0  10  16  15   3   0  0
 0  0   8  15  16  13   0  0  0  1  11      0  0  0  0   9  14   0   0   0  0
 0  0   0   3  11  16   0  0  0  0   5      0  0  0  0   0   1  15   2   0  0
 0  0   5  14   4   0   0  0  0  0  13     12  0  0  0   4  12  14   7   0  0
 0  0  11  16  10   1   0  0  0  4  16  …   8  3  0  0  10  16  16  16  16  6
 0  0   1  11  13  11   7  0  0  0   9      0  0  0  0   1  13   5   0   0  0
 0  0   8  10   8   7   2  0  0  1  15      0  0  0  0   4  13   8   0   0  0
 0  0  15   2  14  13   2  0  0  0  16      0  0  0  0  10  12   5   0   0  0
 0  0   3  13  13   2   0  0  0  6  16     12  0  0  0   3  15  11   6   0  0
 0  0   6  14  14  16  16  8  0  0   7  …   0  0  0  0  10  12   0   0   0  0
 0  0   0   3  16  11   1  0  0  0   0      0  0  0  0   0   2  14  14   1  0
 0  0   0   4  13  16  16  3  0  0   8  

Here is the actual digit of the training set, stored in the 65th column of the training set.

In [67]:
digit = traindata[:, end]
digit

3823-element Array{Int64,1}:
 0
 0
 7
 4
 6
 2
 5
 5
 0
 8
 7
 1
 9
 ⋮
 1
 4
 4
 8
 9
 3
 9
 9
 4
 6
 6
 7

This computes the logpdf of each of the rows of X (each observation).

In [4]:
eachlogpdf = dirmult_logpdf(X', ones(64))

3823-element Array{Float64,1}:
 -165.188
 -176.23 
 -167.774
 -165.564
 -157.79 
 -176.071
 -158.423
 -159.258
 -174.302
 -178.407
 -171.294
 -169.383
 -175.753
    ⋮    
 -160.49 
 -158.633
 -156.935
 -171.975
 -177.638
 -169.735
 -158.423
 -159.465
 -159.877
 -169.735
 -173.149
 -155.411

Note that each observation is independent, so the overall log likelihood is the sum of each of the log pdf's in the `eachlogpdf` vector above.

In [5]:
Log_Likelihood = sum(eachlogpdf)

-638817.993292528

## Q5

Derive the score function $\nabla L(\alpha)$, observed information matrix $-d^2L(\alpha)$, and Fisher information matrix $\mathbf{E}[-d^2L(\alpha)]$ for the Dirichlet-multinomial distribution.

Comment on why Fisher scoring method is inefficient for computing MLE in this example.

## Q5 Solution:

>Let $\psi(x) = \frac{d}{dx}[ \ln \Gamma(x)]$ be the digamma function of x.
Let $\psi_1(x) = \frac{d}{dx}[\psi(x)]$ be the trigamma function of x.

>The Score function, $\nabla L(\alpha) = \frac{d}{d\alpha_j}L(\alpha) = 0 + \sum_{i=1}^n[\psi(\alpha_j + x_{ij}) - \psi(\alpha_j)] - \sum_{i=1}^n[\psi(|\alpha| + |x_i|) - \psi(|\alpha|)]$


>The observed information matrix $-d^2L(\alpha)$ has different forms on the diagonals and on the off diagonals.
<br>
>On the diagonals (where i = j):
$$-\cfrac{\partial^2}{\partial\alpha_i\alpha_j}L(\alpha) = \sum_{i=1}^n[\psi_1(\alpha_j) - \psi_1(\alpha_j + x_{ij})] - \sum_{i=1}^n[\psi_1(|\alpha|) - \psi_1(|\alpha| + |x_i|)]$$
<br>


>On the off diagonals (where i $\ne$ j)
$$-\cfrac{\partial^2}{\partial\alpha_i\alpha_j}L(\alpha) = - \sum_{i=1}^n[\psi_1(|\alpha|) - \psi_1(|\alpha| + |x_i|)]$$

>Now we can see that the observed information matrix has a special structure of a diagonal plus an easy low-rank matrix which is rank 1.
$$
-\cfrac{\partial^2}{\partial\alpha_i\alpha_j}L(\alpha) = \mathbf{D} - c\mathbf{1}\mathbf{1}^T
$$

$$
d_j = \sum_{i=1}^n\Big[\psi_1(\alpha_j) - \psi_1(\alpha_j + x_{ij})\Big]
$$

$$
c = \sum_{i=1}^n\Big[\psi_1(|\alpha|) - \psi_1(|\alpha| + |x_i|) \Big]
$$

<br>
>The Fisher information matrix $\mathbf{E}[-d^2L(\alpha)]$ is difficult to compute, which highlights why Fisher scoring method is inefficient for computing MLE in this example. 

$$
-E\Big[\cfrac{\partial^2}{\partial\alpha_i\alpha_j}L(\alpha) \Big] = E[\mathbf{D} - c\mathbf{1}\mathbf{1}^T]
$$

$$
E[d_j] = \sum_{i = 1}^n\sum_{k=0}^{|x_i|-1}\frac{P(X_{ij} > k)}{(\alpha_j + k)^2}
$$

Where $X_{ij}$ is the $j^{th}$ element of the $i^{th}$ observation of $X$, so it is a beta binomial distribution
and you have to evaluate the beta binomial distribution percentile function to get the tail probabilities for each observation.

Since finding the expected Fisher Information is difficult, it is much easier to ensure positive definiteness by tweaking specific entries of the observed information matrix.

## Q6

What structure does the observed information matrix possess that can facilitate the evaluation of the Newton direction? Is the observed information matrix always positive definite? What remedy can we take if it fails to be positive definite? (Hint: HW1 Q6.)

## Q6 Solution:

No, the observed information matrix is not always positive definite. Some remedies we can make to it involve taking the expectation, resulting in swapping out the observed information matrix for the expected information matrix (Fisher Scoring). However, this remedy may not be the best case in this scenario, as taking expectation of the observed information matrix may be difficult. Thus, we use the Sherman Morrison formula from HW 1 to use the special structure of the observed information matrix, a diagonal plus a rank one update. 

>The observed information matrix has a special structure, a diagonal matrix plus a low rank matrix so we can use the Woodbury formula for the evaluation of the Newton direction. 

>In the case where the matrix fails to be positive definite, we make it become positive definite by manually tweaking the values.

>From the Sherman-Morrison formula and the determinant formula from HW1:

$$
det(\mathbf{D} - c\mathbf{1}\mathbf{1}^T) = det(\mathbf{D})det(\mathbf{I} + c\mathbf{1}^T\mathbf{D}^{-1}\mathbf{1})
$$

$$
= (\prod_{i = 1}^d d_j)(1 - c \sum^d_{j=1}d_j^{-1})
$$
> In order to have a positive definite matrix, the diagonals and the determinant must be positive! So we check that:
$(1 - c \sum^d_{j=1}d_j^{-1}) > 0$


$$
1 > c\sum^d_{j=1}d_j^{-1} \implies c < \cfrac{1}{\sum^d_{j=1}d_j^{-1}}
$$


>We find that the observed information matrix is positive definite when $c < \cfrac{1}{\sum_{i = 1}^d d_j^{-1}}$.

## Q7

Discuss how to choose a good starting point. Implement this as the default starting value in your function below. (Hint: Method of moment estimator may furnish a good starting point.)

## Q7 Solution:

If we start with a smart estimate of $ \alpha$ then the algorithms will converge much faster! So we can take the Method of Moments estimate to initialize the values of alpha. Let $\mathbf{p} = (p_1,\ldots, p_d)$ follow a Dirichlet distribution with parameter vector $\alpha = (\alpha_1,\ldots, \alpha_d)$, $\alpha_j>0$. Then, we can begin by noting the first and second moments of this distribution. 

>The first moment of the Dirichlet Distribution is:
$$E[P_j] = \frac{\alpha_j}{|\alpha|}, 1 \le j \le d$$
<br>

>The second moment of the Dirichlet Distribution is:
$$E[P_j^2] = \frac{\alpha_j(\alpha_j + 1)}{|\alpha|(|\alpha| + 1)}, 1 \le j \le d$$
<br>

Since we know $E[P_j] = \frac{\alpha_j}{|\alpha|}, 1 \le j \le d,$ we know that $\alpha_j = E[P_j]*|\alpha|.$ Now we will use these two moments to first find the value of $\hat|\alpha|$ to find a good estimate for the starting value $\hat\alpha_j$
<br>

>Let $Z = \sum_{j=1}^d \frac{E[P_j^2]}{E[P_j]},$ then we have:

$$Z = \sum_{j = 1}^d \cfrac{E(p_j^2)}{E(p_j)} = \sum_{j = 1}^d \cfrac{\frac{\alpha_j(\alpha_j + 1)}{|\alpha|(|\alpha|+ 1)}}{\frac{\alpha_j}{|\alpha|}} = \sum_{j=1}^d \frac{\alpha_j + 1}{|\alpha| + 1} = \frac{|\alpha| + \sum_{j=1}^d 1}{|\alpha| + 1} =  \cfrac{|\alpha| + d}{|\alpha| + 1}$$
<br>
>Now we want to solve for $|\alpha|$:
<br>
$$ Z(|\alpha| + 1) = |\alpha| + d$$
<br>
$$Z|\alpha| + Z = |\alpha| + d$$
<br>
$$Z|\alpha| - |\alpha| = d - Z$$
<br>
$$|\alpha|(Z - 1) = d - Z$$
<br>
$$|\alpha| = \frac{d - Z}{Z - 1}$$
<br>

> Recall that we are estimating Z by $\hat{Z}$:

$$\hat{Z} = \sum_{j=1}^d\cfrac{{(\sum_i^n\frac{x_{ij}}{|x_i|}})^2}{(\sum_i^n\frac{x_{ij}}{|x_i|})}$$
<br>

>Now we can estimate $\hat{|\alpha|}$ using the estimates for $\hat{Z}$:
<br>

$$\hat{|\alpha|} = \frac{d - \hat{Z}}{\hat{Z} - 1}
$$
<br>

$$ = \frac{d - \sum_{j=1}^d\cfrac{{(\sum_i^n\frac{x_{ij}}{|x_i|}})^2}{(\sum_i^n\frac{x_{ij}}{|x_i|})}}{\sum_{j=1}^d\cfrac{{(\sum_i^n\frac{x_{ij}}{|x_i|}})^2}{(\sum_i^n\frac{x_{ij}}{|x_i|})} - 1}
$$
<br>

>Now we recall that $E[P_j] = \frac{\alpha_j}{|\alpha|}, 1 \le j \le d,$ which means $\alpha_j = E[P_j]*|\alpha|.$ Using our method of moments estimate of $\hat{|\alpha|}$, we get:

$$\hat{\alpha_j} = E[P_j]*\hat{|\alpha|}
$$
<br>
$$ = \frac{(\sum_i^n\frac{x_{ij}}{|x_i|})}{n}*\hat{|\alpha|}
$$
<br>
$$ = \frac{(\sum_i^n\frac{x_{ij}}{|x_i|})}{n}*\frac{d - \sum_{j=1}^d\cfrac{{(\sum_i^n\frac{x_{ij}}{|x_i|}})^2}{(\sum_i^n\frac{x_{ij}}{|x_i|})}}{\sum_{j=1}^d\cfrac{{(\sum_i^n\frac{x_{ij}}{|x_i|}})^2}{(\sum_i^n\frac{x_{ij}}{|x_i|})} - 1}
$$

Now we will implement this into Julia code, and find the Method of Moments estimator for the initial values of alpha in the function `initial_alpha` below.

In [7]:
## code for finding initial alpha
function initial_alpha(X::Matrix{Int})
    n, d = size(X)
    alpha_0 = zeros(d)#creates 64x1 matrix
    tolerance = 0.00001
    Z_hat = 0
    Z_numerator = 0
    sum_Xcol = sum(X, 1)
    sum_Xrow = sum(X, 2)
    
    for j in 1:d
        Z_numerator = 0 #reset each value
        if sum_Xcol[j] < tolerance #check to see if there is data in the column
            continue
        end
        
        for i in 1:n #for each column
            Z_numerator += (X[i, j] / sum_Xrow[i])^2
            alpha_0[j] += (X[i, j] / sum_Xrow[i])
        end
        
        Z_hat += (Z_numerator / alpha_0[j]) #calculating Zhat
        alpha_0[j] /= n #divide by n in front of the sigma

    end
    alpha_0 *= ((d - Z_hat) / (Z_hat - 1))
    return alpha_0
end

alpha_0 = initial_alpha(X)

64-element Array{Float64,1}:
 0.0        
 0.0714198  
 1.30543    
 2.83281    
 2.75176    
 1.32356    
 0.336655   
 0.0343316  
 0.000461199
 0.463798   
 2.52787    
 2.8072     
 2.53299    
 ⋮          
 2.33097    
 2.20214    
 0.894695   
 0.0353529  
 5.71236e-5 
 0.066766   
 1.39754    
 2.8661     
 2.744      
 1.59808    
 0.506003   
 0.0475158  

## Q8

Write a function for finding MLE of Dirichlet-multinomial distribution given iid observations $\mathbf{x}_1,\ldots,\mathbf{x}_n$, using the Newton's method. 

## Q8 Solution:

Below, I wrote a function, `dirmult_newton(X)`, that computes the MLE of the Dirichlet-Multinomial distribution through Newton's Method.

In [83]:
"""
    dirmult_newton(X)

Find the MLE of Dirichlet-multinomial distribution using Newton's method.

# Argument
* `X`: an `n`-by-`d` matrix of counts; each column is one data point.

# Optional argument  
* `alpha0`: a `d` vector of starting point (optional). 
* `maxiters`: the maximum allowable Newton iterations (default 100). 
* `tolfun`: the tolerance for  relative change in objective values (default 1e-6). 

# Output
* `maximum`: the log-likelihood at MLE.   
* `estimate`: the MLE. 
* `gradient`: the gradient at MLE. 
* `hessian`: the Hessian at MLE. 
* `se`: a `d` vector of standard errors. 
* `iterations`: the number of iterations performed.
"""
function dirmult_newton(X) 
    maxiters = 100
    tolfun = 1e-6
    orign, origd = size(X)
    #first reduce to where the obs are 0
    nonzeroobs = find(sum(X, 1) .> 0.01) 
    #remove those rows in X
    X = copy(X[:, nonzeroobs])
    #Xt = copy(X.')
    # set default starting point from Q7
    α0 = initial_alpha(X)
    rowsumX = sum(X, 2)
    n, d = size(X)
    dga = zeros(d)
    score = zeros(d)
    αhat = copy(α0)
    ndir = zeros(d)
    triarow = 0.0
    logl = 0.0
    loglold = sum(dirmult_logpdf(X', αhat))
    # Newton loop
    for iter in 1:maxiters
        rowsumα = sum(αhat)
        score = zeros(d)       
        # evaluate gradient (score)
        dga = digamma.(αhat)
        dgarow = digamma.(rowsumα)
        for j in 1:d
            for i in 1:n
                score[j] += digamma.(αhat[j] + X[i, j]) - dga[j] - digamma.(rowsumα + rowsumX[i]) + dgarow
            end
        end
        # approximated observed information matrix

        D = zeros(d)
        c = 0.0
        triga = trigamma.(αhat)
        for j = 1:d
            for i = 1:n
                D[j] += -trigamma.(αhat[j] + X[i, j]) + triga[j]
                c += -trigamma.(rowsumα + rowsumX[i]) + triarow
            end
        end

        #need to make sure that the condition for c holds for it to be p.d.
        invD = (1 ./ D)
        if !(c < (1 / (sum(invD))))
            c = (1 / sum(invD)) * 0.95
        end

        # compute Newton's direction
        #use Sherman-Morrison inverse formula
        #first part is invD * score
        #second is 1 / (1 - c*sum(Dinv)) * Dinv' * Dinv * c

        for j in 1:d
            ndir[j] = invD[j] * score[j]
        end
        ndir += (((1/c - sum(invD)) * (invD * invD')) * score)

        # line search loop
        step = 1
        for i in 1:d
            if ndir[i] < 0
                step = min((αhat[i] / abs(ndir[i])) * 0.95, step)
            end
        end        
        αupdate = 0.0
        loglupdate = 0.0
        for lsiter in 1:10
            αupdate = αhat + step * ndir
            loglupdate = sum(dirmult_logpdf(X', αupdate))
            # step halving
            if (loglupdate < loglold)
                step = step / 2
            end
        end
        αhat = copy(αupdate)
        logl = copy(loglupdate)

        # check convergence criterion
        if abs(logl - loglold) < (tolfun * (abs(loglold) + 1))
            #println(iter)
            break;
        end
        loglold = copy(logl)  
    end

    # compute logl, gradient, Hessian from final iterate 

    #logl already computed. 
    rowsumα = sum(αhat)
    score = zeros(d)       
    # evaluate gradient (score)
    dga = digamma.(αhat)
    dgarow = digamma.(rowsumα)
    for j in 1:d
        for i in 1:n
            score[j] += digamma.(αhat[j] + X[i, j]) - dga[j] - digamma.(rowsumα + rowsumX[i]) + dgarow
        end
    end

    #evaluating the observed info matrix D is the diagonal and c is the off constant. 
    D = zeros(d)
    c = 0.0
    triga = trigamma.(αhat)
    triarow = trigamma.(rowsumα)
    for j = 1:d
        for i = 1:n
            D[j] += -trigamma.(αhat[j] + X[i, j]) + triga[j]
            c += -trigamma.(rowsumα + rowsumX[i]) + triarow
        end
    end

    Hessian = -1 * (Diagonal(D) + (c * ones(d, d)))
    
    αestimate = zeros(origd)
    αestimate[nonzeroobs] = αhat[1:d]
    return αestimate, logl, Hessian
end

dirmult_newton

It converges after 19 iterations woohoo! Note: Newton's method is highly popular because it converges so quickly.

In [84]:
αestimate_out, logl_out, Hessian_out = dirmult_newton(X)

([0.0, 0.0808195, 0.893214, 2.09767, 2.01352, 0.76266, 0.146161, 0.0151217, 0.000389089, 0.300931  …  0.458806, 0.0272095, 0.000129644, 0.0653641, 0.917977, 2.10699, 1.93648, 0.943011, 0.231207, 0.0277544], -472075.1644846997, [-95363.0 -4279.59 … -4279.59 -4279.59; -4279.59 -9466.99 … -4279.59 -4279.59; … ; -4279.59 -4279.59 … -29642.4 -4279.59; -4279.59 -4279.59 … -4279.59 -2.73177e5])

Note the Hessian is negative, which is good! Since we want the Negative Hessian (Information Matrix) to be positive definite.

In [85]:
Hessian_out

62×62 Array{Float64,2}:
 -95363.0   -4279.59  -4279.59  …  -4279.59   -4279.59  -4279.59     
  -4279.59  -9466.99  -4279.59     -4279.59   -4279.59  -4279.59     
  -4279.59  -4279.59  -6216.82     -4279.59   -4279.59  -4279.59     
  -4279.59  -4279.59  -4279.59     -4279.59   -4279.59  -4279.59     
  -4279.59  -4279.59  -4279.59     -4279.59   -4279.59  -4279.59     
  -4279.59  -4279.59  -4279.59  …  -4279.59   -4279.59  -4279.59     
  -4279.59  -4279.59  -4279.59     -4279.59   -4279.59  -4279.59     
  -4279.59  -4279.59  -4279.59     -4279.59   -4279.59  -4279.59     
  -4279.59  -4279.59  -4279.59     -4279.59   -4279.59  -4279.59     
  -4279.59  -4279.59  -4279.59     -4279.59   -4279.59  -4279.59     
  -4279.59  -4279.59  -4279.59  …  -4279.59   -4279.59  -4279.59     
  -4279.59  -4279.59  -4279.59     -4279.59   -4279.59  -4279.59     
  -4279.59  -4279.59  -4279.59     -4279.59   -4279.59  -4279.59     
      ⋮                         ⋱                 ⋮               

## Q9

Read in `optdigits.tra`, the training set of 3823 handwritten digits. Find the MLE for the subset of digit 0, digit 1, ..., and digit 9 separately using your function. 

## Q9 Solution:

Each column represents each digit from 0-9. For each digit, there are 64 estimated paramaters.

In [46]:
traindata = readcsv("optdigits.tra", Int);

In [65]:
using Distributions
α_MLE = zeros(64, 10)

for digit in 0:9
data = X[find(traindata[:, end] .== digit), :]
    α_MLE[:, digit + 1] = dirmult_newton(data)[1]
end

α_MLE

64×10 Array{Float64,2}:
 0.0        0.0         0.0         …  0.0         0.0         0.0       
 0.027153   0.00806214  0.385797       0.138526    0.0870453   0.0760361 
 3.24571    0.517296    3.77577        2.46692     2.45862     1.51742   
 9.58928    2.11559     5.18795        4.99597     5.7371      3.67141   
 7.80525    3.08047     2.4839         5.38341     5.48944     3.70421   
 1.58007    1.33097     0.308254    …  4.40639     2.38305     1.50018   
 0.0458468  0.138462    0.0156759      1.69695     0.197419    0.29645   
 0.0        0.0         0.0            0.17729     0.0         0.0158601 
 0.0        0.0         0.0            0.0         0.00217486  0.0       
 0.676543   0.0575503   1.71602        0.252583    1.00779     0.662149  
 9.38799    1.04839     5.44015     …  3.13688     6.00204     3.98765   
 9.49482    3.2662      4.70215        3.94263     4.48835     3.32224   
 8.83415    4.13715     4.39493        4.11967     4.01953     3.26306   
 ⋮            

## Q10

As $\alpha / |\alpha| \to \mathbf{p}$, the Dirichlet-multinomial distribution converges to a multinomial with parameter $\mathbf{p}$. Therefore multinomial can be considered as a special Dirichlet-multinomial with $|\alpha|=\infty$. Perform a likelihood ratio test (LRT) whether Dirichlet-multinomial offers a better fit than multinomial for digits 0, 1, ..., 9 respectively.

## Q10 Solution:

First, we subset the training data for each digit, and get the maximum log likelihood of the Dirichlet Multinomial and the Multinomial. Then, I perform a likelihood ratio test (LRT) to determine whether Dirichlet-multinomial offers a better fit than multinomial for digits 0, 1, ..., 9 respectively. Dirichlet-multinomial offers a better for all digits.

In [87]:
using Distributions
logl_dir_mult = zeros(10)
logl_mult = zeros(10)
LRT_pvalue = zeros(10)

for digit in 0:9
data = X[find(traindata[:, end] .== digit), :]
    p = vec(sum(data, 1))
    p = p / sum(p)
    posind = p .> 0
    logl_dir_mult[digit + 1] = dirmult_newton(data)[2]
    for j in 1:size(data, 1)
        logl_mult[digit + 1] += logpdf(Multinomial(Int(sum(data[j, posind])), 
                p[posind]), data[j, posind])
        end
    # compute LRT p-value
    LRT_pvalue[digit + 1] = ccdf(Chisq(1),
        logl_dir_mult[digit + 1] - logl_mult[digit + 1])
    end

Loglikelihood of the Multinomial distribution for digits 0, 1, ..., 9 respectively.

In [88]:
logl_mult

10-element Array{Float64,1}:
 -39592.2
 -54039.2
 -49111.5
 -47089.1
 -57344.1
 -51713.0
 -42597.3
 -49473.0
 -49695.9
 -54577.8

Loglikelihood of the Dirichlet-Multinomial distribution for digits 0, 1, ..., 9 respectively.

In [89]:
logl_dir_mult

10-element Array{Float64,1}:
 -37533.3
 -42179.4
 -39985.5
 -40519.8
 -43488.9
 -41191.5
 -37703.0
 -40304.2
 -43131.2
 -43709.8

P-value from the Likliehood Ratio test comparing the fit of the Dirichlet-multinomial vs. the multinomial for digits 0, 1, ..., 9 respectively. Clearly Dirichlet-Multinomial wins!!!

In [90]:
LRT_pvalue

10-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

## Q11

Now we can construct a simple Bayesian rule for handwritten digits recognition:
$$
	\mathbf{x}	\mapsto \arg \max_k \widehat \pi_k f(x|\widehat \alpha_k).
$$
Here we can use the proportion of digit $k$ in the training set as the prior probability $\widehat \pi_k$. Report the performance of your classifier on the test set of 1797 digits in `optdigits.tes`.

## Q11.Solution:

In order to report the performance of our classifier onthe test of 1797 digits in `optdigits.tes`.

First, we use the training set to estimate the prior probability distribution for p, $\widehat \pi_k$, and store it in the vector `p_hat`. Each of the elements of `p_hat` represents the proportion each digit appears in the training data set.

In [92]:
# read in the training and test data
traindata = readcsv("optdigits.tra", Int)
testdata = readcsv("optdigits.tes", Int)

#first we pre-allocate
p_hat = zeros(10)
n1 = size(traindata, 1) #number of rows or numbers in the training data

# find prior probabilities for each digit in the training set
for digit in 0:9
    p_hat[digit + 1] += sum(traindata[:, 65] .== digit) / n1
end

In [93]:
p_hat

10-element Array{Float64,1}:
 0.0983521
 0.101753 
 0.0993984
 0.101753 
 0.101229 
 0.0983521
 0.0986137
 0.101229 
 0.0993984
 0.0999215

Then, we want to find the Maximum Dirichlet Multinomial logpdf of each digit, by pluggin in the `α_MLE` values for each digit.

In [59]:
DMlogpdf = zeros(10, size(testdata, 1))

#find the maximum logpdf
for digit in 0:9
    DMlogpdf[digit + 1, :] = dirmult_logpdf(testdata[:, 1:64]', α_MLE[:, digit + 1])
end

In [60]:
DMlogpdf

10×1797 Array{Float64,2}:
  -95.1698  -180.967   -179.549  -157.813   …  -161.312  -137.285  -168.489
 -167.935    -90.4242  -119.731  -123.948      -109.194  -135.046  -141.073
 -183.528   -131.541   -136.586  -120.742      -138.909  -161.152  -139.205
 -164.027   -133.974   -157.904   -93.0297     -138.638  -129.132  -136.661
 -141.035   -114.707   -140.391  -150.597      -124.078  -140.22   -155.619
 -142.348   -126.22    -160.841  -112.309   …  -136.663  -129.85   -151.549
 -168.347   -149.028   -153.063  -141.866      -144.654  -164.331  -145.907
 -161.813   -139.3     -Inf      -160.968      -143.385  -162.377  -Inf    
 -148.061   -115.537   -126.78   -125.125      -106.462  -129.631  -125.16 
 -134.604   -131.085   -157.706  -112.109      -134.426  -108.059  -148.108

Now for each digit, we can multiply `DMlogpdf` by the prior distribution of p, `p_hat` to get posterior distribution of each digit in `Posterior`. Then we can get the predicted digit values in the vector, `predicted`.

In [94]:
n2 = size(test_data, 1) #number of rows or numbers in the testing data
Posterior = zeros(10, size(testdata, 1))

# multiply the prior with the logpdf to get the posterior distribution
for j in 1:10
    Posterior[j, :] = DMlogpdf[j, :] * p_hat[j]
end

# predict which digit, based on which has max log likelihood
predicted = zeros(n2)
for i in 1:n2
    predicted[i] = (findmax(probs[:, i], 1)[2][1] - 1)
end

In [95]:
predicted

1797-element Array{Float64,1}:
 0.0
 1.0
 1.0
 3.0
 4.0
 9.0
 6.0
 7.0
 8.0
 9.0
 0.0
 1.0
 2.0
 ⋮  
 7.0
 9.0
 5.0
 4.0
 8.0
 8.0
 4.0
 9.0
 0.0
 8.0
 9.0
 8.0

Now we can compare the true digit value in the last column of the testing data, to the digit predicted from the posterior, and get our classification error and accuracy.

In [96]:
@show error = sum(test_data[:, 65] .!= predicted) / length(predicted)
@show accuracy = 1 - error;

error = sum(test_data[:, 65] .!= predicted) / length(predicted) = 0.11741791875347801
accuracy = 1 - error = 0.8825820812465219
